## Search queries from Topics

### imports

In [8]:
import pandas as pd
from elasticsearch import Elasticsearch
from credentials import username, password

### Read information need from xlm file

In [11]:
xml_filepath = "data/topics-rnd5_covid-complete.xml"
topics = pd.read_xml(xml_filepath)

In [17]:
# set index to number
topics.set_index("number", inplace=True)

In [21]:
topics.head(10)

,query,question,narrative
number,,,
1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...
6,coronavirus test rapid testing,what types of rapid testing for Covid-19 have ...,Looking for studies identifying ways to diagno...
7,serological tests for coronavirus,are there serological tests that detect antibo...,Looking for assays that measure immune respons...
8,coronavirus under reporting,how has lack of testing availability led to un...,Looking for studies answering questions of imp...
9,coronavirus in Canada,how has COVID-19 affected Canada,"seeking data related to infections (confirm, s..."


### Get elasticsearch instance

In [28]:
es = Elasticsearch("http://localhost:9200",
                   basic_auth=(username, password))

In [132]:
query = {
    "aggs": {
        "histogram":{
            "field": "cord_uid"
        }
    }
}


In [133]:
es.search(index="default", aggs=query, size=0)

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [cord_uid] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [75]:
run = "baseline"
with open(f"data/runs/{run}.txt", "w") as f:
    for idx, doc in topics.iterrows():
        q = doc["query"]
        result = es.search(index="default", q=q, size=1000)
        hits = result["hits"]["hits"]

        for rank, hit in enumerate(hits):
            f.write(" ".join([str(idx), str(0), hit["_source"]["cord_uid"], str(rank), str(hit["_score"]), run, "\n"]))